In [35]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

def clean_filename(url):
    filename = url.split('/')[-1].split('?')[0]
    if not filename.lower().endswith(('.png', '.jpeg', '.jpg')):
        filename += '.jpeg'
    return filename

def get_product_info(url, save_path):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Récupère le titre de la page
    page_title = soup.select_one('title').text if soup.select_one('title') else 'No Title'
    
    product_images = soup.find_all('img', {'alt': 'main product photo', 'class': 'gallery-placeholder__image'})
    
    products = []
    
    for img in product_images:
        image_url = img['src']
        img_response = requests.get(image_url, stream=True)
        
        if img_response.status_code == 200:
            filename = clean_filename(image_url)
            filepath = os.path.join(save_path, filename)
            
            with open(filepath, 'wb') as f:
                for chunk in img_response.iter_content(1024):
                    f.write(chunk)
            
            print(f'L\'image a été téléchargée et sauvegardée sous : {filepath}')
            products.append({
                'filename': filename,
                'title': page_title,
            })
        else:
            print('Échec du téléchargement de l\'image.')

    return products


# Fonction pour sauvegarder les données dans un fichier CSV en mode ajout
def save_to_csv(products, save_path, csv_filename="product_images.csv"):
    # Chemin complet du fichier CSV
    csv_path = os.path.join(save_path, csv_filename)
    
    # Crée un DataFrame à partir des produits
    df = pd.DataFrame(products)
    
    # Vérifie si le fichier existe déjà pour décider de l'en-tête
    header = not os.path.exists(csv_path)
    
    # Ajoute les données au fichier CSV, 'a' pour append, ajout à la fin
    df.to_csv(csv_path, mode='a', index=False, header=header, encoding='utf-8')
    print(f'Les données ont été ajoutées dans {csv_path}')


#url = 'https://www.boticinal.com/la-roche-posay-toleriane-sensitive-creme-hydratante-anti-rougeurs-40ml.html'
url = 'https://www.boticinal.com/la-roche-posay-effaclar-gel-moussant-nettoyant-visage-anti-boutons-anti-points-noirs-400ml.html'
save_path = "images"  # Ou spécifiez un chemin de répertoire
products = get_product_info(url, save_path)

# Sauvegarde des informations dans un fichier CSV
save_to_csv(products,"")


L'image a été téléchargée et sauvegardée sous : images/powersante-la-roche-posay-effaclar-gel-moussant-purifiant-400-ml.png
Les données ont été ajoutées dans product_images.csv


In [38]:
import requests
from bs4 import BeautifulSoup

def get_sitemap_urls(sitemap_index_url):
    # Télécharge le contenu de l'index sitemap
    response = requests.get(sitemap_index_url)
    soup = BeautifulSoup(response.content, 'lxml-xml')  # Utilisez 'lxml-xml' pour analyser les fichiers XML avec lxml
    
    # Trouve tous les éléments <loc> qui contiennent les URLs des sitemaps
    sitemap_urls = [loc.text for loc in soup.find_all('loc')]
    
    return sitemap_urls

def get_page_urls(sitemap_urls):
    page_urls = []
    for sitemap_url in sitemap_urls:
        # Télécharge chaque sitemap individuel
        response = requests.get(sitemap_url)
        soup = BeautifulSoup(response.content, 'lxml-xml')
        
        # Extrait les URLs des pages
        page_urls.extend([loc.text for loc in soup.find_all('loc')])
    
    return page_urls

# URL de l'index sitemap
sitemap_index_url = 'https://www.boticinal.com/sitemap.xml'

# Obtenez les URLs des sitemaps
sitemap_urls = get_sitemap_urls(sitemap_index_url)

# Obtenez les URLs des pages à partir des sitemaps individuels
page_urls = get_page_urls(sitemap_urls)

# Affichez ou utilisez les URLs des pages
for url in page_urls:
    print(url)

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml-xml. Do you need to install a parser library?